In [7]:
import os
import json

ground_truth_json_file_path = os.path.join(
    os.environ["CODE"],
    "scripts/07_reproduce_baseline_results/data/ego4d/ego4d_clip_annotations_v3.json",
)
with open(
    ground_truth_json_file_path, "r"
) as ground_truth_json_file_reader:
    ground_truths_dict = json.load(ground_truth_json_file_reader)


In [15]:
import cv2

cap = cv2.VideoCapture("/data/aarslan/ego4d_data/v2/clips/00462c6f-c50f-4005-b3a7-6253fa6e9cc3.mp4")


In [18]:
number_of_frames = 0

cap = cv2.VideoCapture("/data/aarslan/ego4d_data/v2/clips/00462c6f-c50f-4005-b3a7-6253fa6e9cc3.mp4")
success = True
while success:
    success, frame = cap.read()
    number_of_frames += 1
    

In [19]:
number_of_frames

14401

In [17]:
cap.get(cv2.CAP_PROP_FPS)


30.0

In [14]:
ground_truths_dict["00462c6f-c50f-4005-b3a7-6253fa6e9cc3"]

{'video_id': '6bea0b73-ab46-4615-a86e-cb65784dba0b',
 'clip_id': '00462c6f-c50f-4005-b3a7-6253fa6e9cc3',
 'duration': 480.0,
 'parent_start_sec': 0.0,
 'parent_end_sec': 480.0,
 'v_duration': 649.3,
 'fps': 1.8743261974434007,
 'annotations': [{'segment': [385.27911, 391.608],
   'label': 'walk_down_stairs_/_walk_up_stairs',
   'label_id': 30},
  {'segment': [376.39362, 381.521],
   'label': 'use_hammer_/_nail-gun_to_fix_nail',
   'label_id': 34},
  {'segment': [373.9374, 381.085],
   'label': 'use_hammer_/_nail-gun_to_fix_nail',
   'label_id': 34},
  {'segment': [384.81953, 392.004],
   'label': 'walk_down_stairs_/_walk_up_stairs',
   'label_id': 30}],
 'subset': 'train'}

In [23]:
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output, no_update, callback

# Small molecule drugbank dataset
# Source: https://raw.githubusercontent.com/plotly/dash-sample-apps/main/apps/dash-drug-discovery/data/small_molecule_drugbank.csv'
data_path = 'drug-discovery_data_small_molecule_drugbank.csv'

df = pd.read_csv(data_path, header=0, index_col=0)

fig = go.Figure(data=[
    go.Scatter(
        x=df["LOGP"],
        y=df["PKA"],
        mode="markers",
        marker=dict(
            colorscale='viridis',
            color=df["MW"],
            size=df["MW"],
            colorbar={"title": "Molecular<br>Weight"},
            line={"color": "#444"},
            reversescale=True,
            sizeref=45,
            sizemode="diameter",
            opacity=0.8,
        )
    )
])

# turn off native plotly.js hover effects - make sure to use
# hoverinfo="none" rather than "skip" which also halts events.
fig.update_traces(hoverinfo="none", hovertemplate=None)

fig.update_layout(
    xaxis=dict(title='Log P'),
    yaxis=dict(title='pkA'),
    plot_bgcolor='rgba(255,255,255,0.1)'
)

app = Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id="graph-basic-2", figure=fig, clear_on_unhover=True),
    dcc.Tooltip(id="graph-tooltip"),
])


@callback(
    Output("graph-tooltip", "show"),
    Output("graph-tooltip", "bbox"),
    Output("graph-tooltip", "children"),
    Input("graph-basic-2", "hoverData"),
)
def display_hover(hoverData):
    if hoverData is None:
        return False, no_update, no_update

    # demo only shows the first point, but other points may also be available
    pt = hoverData["points"][0]
    bbox = pt["bbox"]
    num = pt["pointNumber"]

    df_row = df.iloc[num]
    img_src = df_row['IMG_URL']
    name = df_row['NAME']
    form = df_row['FORM']
    desc = df_row['DESC']
    if len(desc) > 300:
        desc = desc[:100] + '...'

    children = [
        html.Div([
            html.Img(src=img_src, style={"width": "100%"}),
            html.H2(f"{name}", style={"color": "darkblue", "overflow-wrap": "break-word"}),
            html.P(f"{form}"),
            html.P(f"{desc}"),
        ], style={'width': '200px', 'white-space': 'normal'})
    ]

    return True, bbox, children


if __name__ == "__main__":
    app.run(debug=True)
